In [1]:
%pylab inline

import os
import pandas as pd
from collections import OrderedDict
from astropy.io import fits
from copy import deepcopy as copy
from scipy.optimize import leastsq
from styles import slides

Populating the interactive namespace from numpy and matplotlib


In [2]:
rdir = os.path.expandvars("$HOME/Research/photometric-ew")
flist = sorted([os.path.join(root,file) for root, subs, files in os.walk(rdir+"/data/fits") for file in files if file.endswith(".fit.gz")])

line_ids = [1,2,5,6,10,13,16,17,24,26,27,28,29,30,35,36,37,38,39,40]
with fits.open(flist[0]) as f: line_rwl = f[2].data["restWave"][line_ids]
line_nms = "Lyalpha", "NV", "SiIV", "CIV", "CIII", "MgII", "OII3727", "OII3730", "Hdelta", "Hgamma", "OIII4364", "Hbeta", "OIII4960", "OIII5008", "NII6550", "Halpha", "NII6585", "Li", "SII6728", "SII6733"
band_nms = "u g r i z".split()
fits_nms = [name.split("/")[-1] for name in flist]

rows = OrderedDict(zip(line_nms,line_ids))
ew = OrderedDict(zip(line_nms,([],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[])))
er = OrderedDict(zip(line_nms,([],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[])))

z, mags = [], OrderedDict(zip(band_nms,([],[],[],[],[])))
for fn in flist:
    with fits.open(fn) as f:
        z.append(f[0].header["z"])
        mag = map(eval,f[0].header["MAG"].split())
        for j,kw in enumerate(band_nms):
            mags[kw] += [mag[j]]
        for kw in ew:
            ew[kw] += [f[2].data["ew"][rows[kw]]]
            er[kw] += [f[2].data["ewErr"][rows[kw]]]

In [6]:
table_dat = OrderedDict()
table_dat["FITS"] = fits_nms
table_dat.update(ew)
table_dat.update(mags)
table_dat["redshift"] = z

df_dat = pd.DataFrame(table_dat)
df_dat.to_csv("{}/data/photo-speclines.csv".format(rdir))

table_err = OrderedDict()
table_err["FITS"] = fits_nms
table_err.update(er)

df_err = pd.DataFrame(table_err)
df_err.to_csv("{}/data/photo-speclines_err.csv".format(rdir))